# Safe to eat or deadly poisonous?
### An analysis on mushroom classification by Lorenzo Santolini

#### Code snippet for google colab
This is a little code to import automatically the dataset into google colab. Provide your kaggle's API key (profile section) when file requested

In [2]:

# Little code snippet to import on Google Colab the dataset
'''
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

# Insert here your kaggle API key
from google.colab import files
files.upload()

!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d uciml/mushroom-classification
!unzip mushroom-classification.zip
!ls
'''


'\n!pip install -U -q kaggle\n!mkdir -p ~/.kaggle\n\n# Insert here your kaggle API key\nfrom google.colab import files\nfiles.upload()\n\n!cp kaggle.json ~/.kaggle/\n!kaggle datasets download -d uciml/mushroom-classification\n!unzip mushroom-classification.zip\n!ls\n'

In [3]:
# Constants
PLOTLY_COLORS = ['#140DFF', '#FF0DE2']
COLOR_PALETTE = ['#FF0DE2', '#BD0CE8', '#8B00FF', '#4D0FE8', '#140DFF']
COLORSCALE_HEATMAP = [         [0.0, 'rgb(70,0,252)'], 
                [0.1111111111111111, 'rgb(78,0,252)'], 
                [0.2222222222222222, 'rgb(90,0,252)'], 
                [0.3333333333333333, 'rgb(110,0,248)'], 
                [0.4444444444444444, 'rgb(130,0,238)'], 
                [0.5555555555555556, 'rgb(145,0,228)'], 
                [0.6666666666666666, 'rgb(166,0,218)'], 
                [0.7777777777777778, 'rgb(187,0,213)'], 
                [0.8888888888888888, 'rgb(200,0,202)'], 
                               [1.0, 'rgb(210,0,191)']]
PLOTLY_OPACITY = 0.7
RANDOM_SEED = 11

LOGISTIC_REGRESSION_PARAMS = {
    'clf__solver': ['liblinear'],  # best for small datasets
    'clf__C': [0.01, 0.1, 1, 10, 100], # smaller value, stronger regularization
    'clf__penalty': ['l2', 'l1']
}



## Introduction
This dataset includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family Mushroom. Each species is identified as edible or poisonous. Rows are composed by 23 different fields, each one of them identifying a specific charateristic:

- Class: poisonous=p, edible=e
- Cap-shape: bell=b, conical=c, convex=x, flat=f, knobbed=k, sunken=s
- Cap-surface: fibrous=f, grooves=g, scaly=y, smooth=s
- Cap-color: brown=n, buff=b, cinnamon=c, gray=g, green=r, pink=p, purple=u, red=e, white=w, yellow=y
- Bruises: bruises=t, no=f
- Odor: almond=a, anise=l, creosote=c, fishy=y, foul=f, musty=m, none=n, pungent=p, spicy=s
- Gill-attachment: attached=a, descending=d, free=f, notched=n
- Gill-spacing: close=c, crowded=w, distant=d
- Gill-size: broad=b, narrow=n
- Gill-color: black=k, brown=n, buff=b, chocolate=h, gray=g, green=r, orange=o, pink=p, purple=u,red=e, white=w, yellow=y
- Stalk-shape: enlarging=e, tapering=t
- Stalk-root: bulbous=b, club=c, cup=u, equal=e, rhizomorphs=z, rooted=r, missing=?
- Stalk-surface-above-ring: fibrous=f, scaly=y, silky=k, smooth=s
- Stalk-surface-below-ring: fibrous=f, scaly=y, silky=k, smooth=s
- Stalk-color-above-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
- Stalk-color-below-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
- Veil-type: partial=p, universal=u
- Veil-color: brown=n, orange=o, white=w, yellow=y
- Ring-number: none=n, one=o, two=t
- Ring-type: cobwebby=c, evanescent=e, flaring=f, large=l, none=n, pendant=p, sheathing=s, zone=z
- Spore-print-color: black=k, brown=n, buff=b, chocolate=h, green=r, orange=o, purple=u, white=w, yellow=y
- Population: abundant=a, clustered=c, numerous=n, scattered=s, several=v, solitary=y
- Habitat: grasses=g, leaves=l, meadows=m, paths=p, urban=u, waste=w, woods=d

This analysis was conducted in Python 3.7.1 using Jupyter Notebook allows you to combine code, comments, multimedia, and visualizations in an interactive document — called a notebook, naturally — that can be shared, re-used, and re-worked. In addition, the following packages were used:

- sklearn
- pandas
- numpy
- plotly


In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV

import plotly
import plotly.plotly as py
from plotly.plotly import plot, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff

from scipy.cluster import hierarchy as hc

from imblearn.pipeline import make_pipeline, Pipeline
from imblearn.over_sampling import SMOTE

from prettytable import PrettyTable
from functools import wraps
import time

plotly.tools.set_credentials_file(username='modusV', api_key='OBKKnTR2vYTeKIOKtRU6')


In [ ]:

# Wrapper to calculate functions speed

def watcher(func):
    """
    Decorator for dumpers.
    Shows how much time it
    takes to create/retrieve
    the blob.
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f" ===> took {end-start} seconds")
        return result
    return wrapper


## Dataset analysis and preprocessing
Let's start importing the data:

In [5]:
# Load the dataset
dataset = pd.read_csv("./Input/mushrooms.csv")
# dataset = pd.read_csv("./mushrooms.csv")
print("The dataset has %d rows and %d columns." % dataset.shape)

The dataset has 8124 rows and 23 columns.


Now we will look at the dataset to understand what are the different fields and their types:

In [6]:
# Count number of classes for classification
print(f"There are {dataset['class'].unique().size} different classes:"
      f"\n {dataset['class'].unique().tolist()}")

# Count number of unique data for every column
print(f"Unique values for every field: \n{dataset.nunique()}")


There are 2 different classes:
 ['p', 'e']
Unique values for every field: 
class                        2
cap-shape                    6
cap-surface                  4
cap-color                   10
bruises                      2
odor                         9
gill-attachment              2
gill-spacing                 2
gill-size                    2
gill-color                  12
stalk-shape                  2
stalk-root                   5
stalk-surface-above-ring     4
stalk-surface-below-ring     4
stalk-color-above-ring       9
stalk-color-below-ring       9
veil-type                    1
veil-color                   4
ring-number                  3
ring-type                    5
spore-print-color            9
population                   6
habitat                      7
dtype: int64


In [7]:
# See data types 
print(f"Data types: \n{dataset.head(5)}")

# All columns 
print(", ".join(str(a) for a in dataset.columns))

Data types: 
  class cap-shape cap-surface cap-color bruises odor gill-attachment  \
0     p         x           s         n       t    p               f   
1     e         x           s         y       t    a               f   
2     e         b           s         w       t    l               f   
3     p         x           y         w       t    p               f   
4     e         x           s         g       f    n               f   

  gill-spacing gill-size gill-color   ...   stalk-surface-below-ring  \
0            c         n          k   ...                          s   
1            c         b          k   ...                          s   
2            c         b          n   ...                          s   
3            c         n          n   ...                          s   
4            w         b          k   ...                          s   

  stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
0                      w                      w  

From the above snippet we can notice that the fields are all string values; converting them to numeric values can make our analysis much easier. We will take care of this in the next phase.

## Preprocessing
We need to pre-process our data to 


In [8]:
n_columns_original = len(dataset.columns)
to_drop = [col for col in dataset.columns if dataset[col].nunique() == 1]
dataset.drop(to_drop, axis=1, inplace=True)

print(f"{n_columns_original - len(dataset.columns)} not significant columns have been removed")


#### Handling missing values

1 not significant columns have been removed


In [9]:
# Check if any field is null
if dataset.isnull().any().any():
    print("There are some null values")
else:
    print("There are no null values")

There are no null values


It may seem that we have no missing value from the previous analysis, but if we look better,from the data description we can notice that in the field stalk-root there are some missing values, marked with the question mark; let's count how many of them there are:


In [10]:
print("There are " + str((dataset['stalk-root'] == "?").sum()) + " missing values in stalk-root column")
# df_drop = dataset[dataset['stalk-root'] != "?"]


There are 2480 missing values in stalk-root column


When we find missing values in a dataset, there are some of the approaches that can be considered:

1. Delete all rows containing a missing value
2. Substitute with a constant value that has meaning within the domain, such as 0, distinct from all other values.
3. Substitute with a value from another randomly selected record.
4. Substitute with mean, median or mode value for the column.
5. Substitute with a value estimated by another predictive model.

It is evident from the `dataset.head()` function that our fileds are composed by all string values. Given the fact that we would need to translate in any case every field to a numeric one, to better display them in graphs, a simple approach is to keep the missing data as a peculiar number different from the others, and simply apply the transformation as they were present.

We will use the LabelEncoder from the sklearn library, which allows us to perform this mapping:



In [11]:
def preprocess(dataset):
    mapping = {}  
    d = dataset.copy()
    labelEncoder = LabelEncoder()
    for column in dataset.columns:
        labelEncoder.fit(dataset[column])
        mapping[column] = dict(zip(labelEncoder.classes_, labelEncoder.transform(labelEncoder.classes_)))
        d[column] = labelEncoder.transform(dataset[column])
        
    return d, labelEncoder, mapping

le = 0
pre_data, l_encoder, le_mapping = preprocess(dataset)

# Check mapping
print(le_mapping)

# Check new data
pre_data.head(5)


{'class': {'e': 0, 'p': 1}, 'cap-shape': {'b': 0, 'c': 1, 'f': 2, 'k': 3, 's': 4, 'x': 5}, 'cap-surface': {'f': 0, 'g': 1, 's': 2, 'y': 3}, 'cap-color': {'b': 0, 'c': 1, 'e': 2, 'g': 3, 'n': 4, 'p': 5, 'r': 6, 'u': 7, 'w': 8, 'y': 9}, 'bruises': {'f': 0, 't': 1}, 'odor': {'a': 0, 'c': 1, 'f': 2, 'l': 3, 'm': 4, 'n': 5, 'p': 6, 's': 7, 'y': 8}, 'gill-attachment': {'a': 0, 'f': 1}, 'gill-spacing': {'c': 0, 'w': 1}, 'gill-size': {'b': 0, 'n': 1}, 'gill-color': {'b': 0, 'e': 1, 'g': 2, 'h': 3, 'k': 4, 'n': 5, 'o': 6, 'p': 7, 'r': 8, 'u': 9, 'w': 10, 'y': 11}, 'stalk-shape': {'e': 0, 't': 1}, 'stalk-root': {'?': 0, 'b': 1, 'c': 2, 'e': 3, 'r': 4}, 'stalk-surface-above-ring': {'f': 0, 'k': 1, 's': 2, 'y': 3}, 'stalk-surface-below-ring': {'f': 0, 'k': 1, 's': 2, 'y': 3}, 'stalk-color-above-ring': {'b': 0, 'c': 1, 'e': 2, 'g': 3, 'n': 4, 'o': 5, 'p': 6, 'w': 7, 'y': 8}, 'stalk-color-below-ring': {'b': 0, 'c': 1, 'e': 2, 'g': 3, 'n': 4, 'o': 5, 'p': 6, 'w': 7, 'y': 8}, 'veil-color': {'n': 0, 'o

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,2,7,7,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,2,7,7,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,2,7,7,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,2,7,7,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,2,7,7,2,1,0,3,0,1


Now let's check if there is any column not useful for classification, 
where the values are all the same (zero variance)



In [12]:
# Check new labels
print(pre_data.groupby('class').size())

class
0    4208
1    3916
dtype: int64


We can notice that data have been transformed, and now the labels are represented with a 0/1 integer value. 
Now we can look deeper into some statistical details about the dataset, using the `dataset.describe` command on our pandas DataFrame dataset. The output describes:

- count: number of samples (rows)
- mean: the mean of the attribute among all samples
- std: the standard deviation of the attribute
- min: the minimal value of the attribute
- 25%: the lower percentile
- 50%: the median
- 75%: the upper percentile
- max: the maximal value of the attribute

In [13]:
# Get insights on the dataset
dataset.describe()


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,2,9,2,2,2,12,...,4,4,9,9,4,3,5,9,6,7
top,e,x,y,n,f,n,f,c,b,b,...,s,s,w,w,w,o,p,w,v,d
freq,4208,3656,3244,2284,4748,3528,7914,6812,5612,1728,...,5176,4936,4464,4384,7924,7488,3968,2388,4040,3148


In [14]:
y = dataset["class"].value_counts()
print(y)
class_dict = ["edible", "poisonous"]


e    4208
p    3916
Name: class, dtype: int64


This is the class distribution:

In [92]:
data = [go.Bar(
            x=class_dict,
            y=y,
            marker=dict(
            color=PLOTLY_COLORS),
            opacity=PLOTLY_OPACITY,
    )]

layout = go.Layout(title="Class distribution",
                   autosize=False,
                   width=400,
                   height=400,
                   yaxis=dict(
                        title='N. samples',
                    ),
                   )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='color-bar')


/Users/lorenzosantolini/miniconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead




At this point we can analyze the distribution of our data using a boxplot. 
A boxplot is a standardized way of displaying the distribution of data based on a 
five number summary (“minimum”, first quartile (Q1), median, third quartile (Q3), and “maximum”). 
It can tell you about your outliers and what their values are. 
It can also tell you if your data is symmetrical, how tightly your data is grouped, 
and if and how your data is skewed.
The information that we can find in a box plot are:

- **median** (Q2/50th Percentile): the middle value of the dataset.
- **first quartile** (Q1/25th Percentile): the middle number between the smallest number (not the “minimum”) and the median of the dataset.
- **third quartile** (Q3/75th Percentile): the middle value between the median and the highest value (not the “maximum”) of the dataset.
- **interquartile range** (IQR): 25th to the 75th percentile.
- **outliers** (shown as green circles)
- **maximum**: Q3 + 1.5*IQR
- **minimum**: Q1 -1.5*IQR

It makes no sense showing binary or with few different values fields, so we are going to filter them before plotting.


In [93]:

def create_box(type, data, col, visible=False):
    if type == "edible":
        c = PLOTLY_COLORS[0]
    else:
        c = PLOTLY_COLORS[1]
    return go.Box(
        y = data[col],
        name = type,
        marker=dict(color = c),
        visible=visible,
        opacity=PLOTLY_OPACITY,
    )

edible = pre_data[pre_data["class"] == 0]
poisonous = pre_data[pre_data["class"] == 1]
box_features = [col for col in pre_data.columns if ((col != 'class') and (dataset[col].nunique() > 5))]

active_index = 0
box_edible = [(create_box("edible", edible, col, False) if i != active_index 
               else create_box("edible", edible, col, True)) 
              for i, col in enumerate(box_features)]

box_poisonous = [(create_box("poisonous", poisonous, col, False) if i != active_index 
               else create_box("poisonous", poisonous, col, True)) 
              for i, col in enumerate(box_features)]

data = box_edible + box_poisonous
n_features = len(box_features)
steps = []

for i in range(n_features):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data)],
        label = box_features[i],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    step['args'][1][i + n_features] = True # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(
    active = active_index,
    currentvalue = dict(
        prefix = "Feature: ", 
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
    len=1,
)]

layout = dict(
    sliders=sliders,
    yaxis=dict(
        title='value',
        automargin=True,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='box_slider')



From the boxplot above, we can see that the colour and the shape of the 
cap are not an effective parameter to decide whether a mushroom is poisonous or edible, 
because their plots are very similar (same median and very close distribution). 
The gill color instead, is more significant; 



Let's investigate the correlation between variables:


In [94]:
correlation_matrix = pre_data.corr(method='pearson')

trace = go.Heatmap(
    z=correlation_matrix.values.tolist(), 
    x=correlation_matrix.columns, 
    y=correlation_matrix.columns, 
    colorscale=COLORSCALE_HEATMAP,
    opacity=0.95,
    zmin=-1,
    zmax=1)
    

data=[trace]

layout = go.Layout(
    title='Heatmap of columns correlation',
    autosize=False,
    width=850,
    height=700,
    yaxis=go.layout.YAxis(automargin=True),
    xaxis=dict(tickangle=40),
    margin=go.layout.Margin(l=0, r=200, b=200, t=80)
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='labelled-heatmap4')



TODO: Give some impressions on the heatmap


In [95]:

def create_hist(type, data, col, visible=False):
    if type == "edible":
        c = PLOTLY_COLORS[0]
    else:
        c = PLOTLY_COLORS[1]
    return go.Histogram(
        x = data[col],
        name = type,
        marker=dict(color = c),
        visible=visible,
        opacity=PLOTLY_OPACITY,
    )

hist_features = [col for col in pre_data.columns if (col != 'class')]

active_index = 0
hist_edible = [(create_hist("edible", edible, col, False) if i != active_index 
               else create_hist("edible", edible, col, True)) 
              for i, col in enumerate(hist_features)]

hist_poisonous = [(create_hist("poisonous", poisonous, col, False) if i != active_index 
               else create_hist("poisonous", poisonous, col, True)) 
              for i, col in enumerate(hist_features)]

total_data = hist_edible + hist_poisonous
n_features = len(hist_features)
steps = []

for i in range(n_features):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(total_data)],
        label = hist_features[i],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    step['args'][1][i + n_features] = True # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(
    active = active_index,
    currentvalue = dict(
        prefix = "Feature: ", 
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
)]

layout = dict(
    sliders=sliders,
    yaxis=dict(
        title='value',
        automargin=True,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = dict(data=total_data, layout=layout)
py.iplot(fig, filename='hist_slider')


TODO give impressions on the histogram


In [96]:

X = np.random.rand(10, 10)
names = pre_data.columns
inverse_correlation = 1 - abs(pre_data.corr()) # This is the 'dissimilarity' method
fig = ff.create_dendrogram(inverse_correlation.values, 
                           labels=names, 
                           colorscale=PLOTLY_COLORS, 
                           linkagefun=lambda x: hc.linkage(x, 'average'))

fig['layout'].update(dict(
    title="Dendrogram of correlation among features",
    width=800, 
    height=600,
    xaxis=dict(
        title='Features',
    ),
    yaxis=dict(
        title='Distance',
        
    ),
))
iplot(fig, filename='dendrogram_corr_clustering')


TODO: comment the graph, every feature is useful

Now we standardize the data 


In [19]:

from sklearn.preprocessing import StandardScaler


drop_data = pre_data[pre_data['stalk-root'] != le_mapping['stalk-root']['?']]

y_data = pre_data['class']
y_drop_data = drop_data['class']

X_pre_data = pre_data.drop(['class'], axis=1)
X_drop_data = drop_data.drop(['class'], axis=1)

scaler = StandardScaler()

X_scaled_data = scaler.fit_transform(X_pre_data)
X_scaled_drop_data = scaler.fit_transform(X_drop_data)



/Users/lorenzosantolini/miniconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning:

Data with input dtype int64 were all converted to float64 by StandardScaler.

/Users/lorenzosantolini/miniconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning:

Data with input dtype int64 were all converted to float64 by StandardScaler.



(5644, 21)

TODO: introduce PCA


In [17]:

from sklearn.decomposition import PCA


pca = PCA(random_state=RANDOM_SEED)
projected_data = pca.fit_transform(X_scaled_data)

tot_var = np.sum(pca.explained_variance_)
ex_var = [(i / tot_var) * 100 for i in sorted(pca.explained_variance_, reverse=True)]
cum_ex_var = np.cumsum(ex_var)




In [22]:
cum_var_bar = go.Bar(
    x=list(range(1, len(cum_ex_var) + 1)), 
    y=ex_var,
    name="Variance of each component",
    marker=dict(
        color=PLOTLY_COLORS[0],
    ),
    opacity=PLOTLY_OPACITY
)
variance_line = go.Scatter(
    x=list(range(1, len(cum_ex_var) + 1)),
    y=cum_ex_var,
    mode='lines+markers',
    name="Cumulative variance",
    marker=dict(
        color=PLOTLY_COLORS[1],
    ),
    opacity=PLOTLY_OPACITY,
    line=dict(
        shape='hv',
    ))
data = [cum_var_bar, variance_line]
layout = go.Layout(
    title='Individual and Cumulative Explained Variance',
    autosize=True,
    yaxis=dict(
        title='Explained variance (%)',
    ),
    xaxis=dict(
        title="Principal components",
        dtick=1,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='basic-bar')


/Users/lorenzosantolini/miniconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



From the graph we can see that the first 9 components retain almost 80% of 
total variance, while last 5 not even 2%. We then choose to select first 
nine of them.

This is the reduced dataset:


In [18]:

n_comp = 9
pca.components_ = pca.components_[:n_comp]
reduced_data = pca.inverse_transform(projected_data)
X_df_reduced = pd.DataFrame(reduced_data, columns=["PC#%d" % (x + 1) for x in range(n_comp)])


ValueError: shapes (8123,22) and (9,22) not aligned: 22 (dim 1) != 9 (dim 0)

In [23]:

from sklearn.cluster import KMeans

N=pre_data.values
pca = PCA(n_components=2)
x = pca.fit_transform(N)

kmeans = KMeans(n_clusters=2, random_state=RANDOM_SEED)
X_clustered = kmeans.fit_predict(N)
print(kmeans.score)

ed_idx = np.where(X_clustered == 0)
po_idx = np.where(X_clustered == 1)

p1 = go.Scatter(
    x=np.take(x[:,0], indices=ed_idx)[0],
    y=np.take(x[:,1], indices=ed_idx)[0],
    mode='markers',
    name="Edible",
    marker=dict(
        color=PLOTLY_COLORS[0],
    ),
    opacity=PLOTLY_OPACITY)

p2 = go.Scatter(
    x=np.take(x[:,0], indices=po_idx)[0],
    y=np.take(x[:,1], indices=po_idx)[0],
    mode='markers',
    name="Poisonous",
    marker=dict(
        color=PLOTLY_COLORS[1],
    ),
    opacity=PLOTLY_OPACITY)
    

data = [p1, p2]

layout = go.Layout(
    title='Data clustered using first two components',
    autosize=True,
    yaxis=dict(
        title='Second component',
    ),
    xaxis=dict(
        title="First component",
        dtick=1,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='clusters-scatter')


<bound method KMeans.score of KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=11, tol=0.0001, verbose=0)>


/Users/lorenzosantolini/miniconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead




TODO: give some impressions on the clustered data

Using K-means we are able to separate two classes using the two components with maximum variance.


Now, before starting the classification phase, let's see what kind of pre-processed
data it is better to use to achieve the best classification possible.
Due to the fact that our dataset is pretty small, probably the dimensionality reduction
using PCA is not necessary.

We are going to compare results of a classification method on the different datasets.
In this way we can choose the one to pick for the 
next phase. The  current versions of the dataset are:

- Full dataset
- Dataset reduced using first 9 principal components
- Full dataset with missing values removed (question marks in the stalk-root field)

Our dataset is pretty balanced, so we do not need any over or under-sampling
technique. If we will perform poorly in classification, we could try to use
ensemble learning methods.
Let's start with splitting the datasets in train and test.

Bagging, boosting and ensemble learning models
> https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-for-ensemble-models/
> MSMOTE
> Over-Under sampling 
> SMOTE


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X_scaled_data, y_data, test_size=0.15, random_state=RANDOM_SEED)
X_train_pc, X_test_pc, y_train_pc, y_test_pc = train_test_split(X_df_reduced, y_data, test_size=0.15, random_state=RANDOM_SEED)
X_train_drop, X_test_drop, y_train_drop, y_test_drop = train_test_split(X_scaled_drop_data, y_drop_data, test_size=0.15, random_state=RANDOM_SEED)


The first method used will be Logistic Regression, and we will tune its parameters
using a grid search cross validation. Let's start defining the functions
that we are going to use:


In [ ]:

def print_gridcv_scores(grid_search, n=5):
    
    if not hasattr(grid_search, 'best_score_'):
        raise KeyError('grid_search is not fitted.')
    
    t = PrettyTable()
    
    print("Best grid scores on validation set:")
    indexes = np.argsort(grid_search.cv_results_['mean_test_score'])[::-1][:n]
    means = grid_search.cv_results_['mean_test_score'][indexes]
    stds = grid_search.cv_results_['std_test_score'][indexes]
    params = np.array(grid_search.cv_results_['params'])[indexes]
    
    for mean, std, params in zip(means, stds, params):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
        
        
@watcher
def param_tune_grid_cv(clf, params, kf, X_train, y_train, cv):
    grid_search = GridSearchCV(estimator=clf, 
                               param_grid=params, 
                               cv=kf, 
                               n_jobs=-1,       # Use all processors
                               scoring='f1',    # Use f1 metric for evaluation
                               return_train_score=True)
    grid_search.fit(X_train, y_train)
    return grid_search
    

def score(clfs, datasets):
    scores = []
    for c, (X_test, y_test) in zip(clfs, datasets):
        scores.append(c.score(X_test, y_test))
    return scores
